In [2]:
#all da imports
import pandas as pd

In [ ]:
# each row is a festival screening of a film (so films shown multiple times have multiple rows)
fest_long = pd.read_csv("data/1_film-dataset_festival-program_long.csv")
# each row is a film, where the data for the first festival it was shown at listed (only one row per film)
fest_wide = pd.read_csv("data/1_film-dataset_festival-program_wide.csv")
#  each row is an award of a given film, including nominations and wins (so films with multiple awards have multiple rows)
imdb_awards = pd.read_csv("data/3_imdb-dataset_awards_long.csv")
# merge fest_wide and fest_long on "unique.id", keeping all rows from fest_wide, adding the new festivals from fest_long
fest_combined = fest_wide.merge(fest_long, on="unique.id", how="left")

fest_combined.head(20)


,unique.id,imdb.id,title.mixed,prod.year,length.min,length,prod.country.1.en,prod.country.2.en,prod.country.3.en,prod.country.4.en,...,competition,genre,fest.first,fest.sect.first,sample.year.first,fest.show.delay.3year.and.more,film.id,sample.year,fest,fest.sect
0,1,tt2917506,a story of children and film,2013.0,101.0,41 min. or longer,United Kingdom,NaN,NaN,NaN,...,0,other documentary,CANNES,Cannes Classics - Documentaries About Cinema,2013,0.0,f2081,2013,CANNES,Cannes Classics - Documentaries About Cinema
1,10,tt2852460,bends,2013.0,96.0,41 min. or longer,China,NaN,NaN,NaN,...,0,other fiction,TIFF,discovery: 2012,2012,0.0,tffnew391,2012,TIFF,discovery: 2012
2,10,tt2852460,bends,2013.0,96.0,41 min. or longer,China,NaN,NaN,NaN,...,0,other fiction,TIFF,discovery: 2012,2012,0.0,f2023,2013,CANNES,Un Certain Regard
3,100,tt0057494,a legend or was it?,1963.0,83.0,41 min. or longer,Japan,NaN,NaN,NaN,...,0,other fiction,BERLINALE,Forum,2013,1.0,f1160,2013,BERLINALE,Forum
4,1000,tt0032445,somewhere in the netherlands,1940.0,86.0,41 min. or longer,Netherlands,NaN,NaN,NaN,...,0,other fiction,BERLINALE,Retrospective,2013,1.0,f1285,2013,BERLINALE,Retrospective
5,1001,tt3613002,song,2013.0,18.0,40 min. or shorter,United States,NaN,NaN,NaN,...,0,experimental fiction,TIFF,Wavelength,2013,0.0,f3255,2013,TIFF,Wavelength
6,1002,tt3003858,song from the forest,2013.0,97.0,41 min. or longer,Germany,NaN,NaN,NaN,...,1,other documentary,IDFA,IDFA Competition for Feature-Length Documentary,2013,0.0,f6232,2013,IDFA,IDFA Competition for Feature-Length Documentary
7,1003,tt2408026,soongava—dance of the orchids,2012.0,85.0,41 min. or longer,Nepal,NaN,NaN,NaN,...,0,other fiction,FRAMELINE,"World Cinema, Queer Asian Cinema",2013,0.0,f5201,2013,FRAMELINE,"World Cinema, Queer Asian Cinema"
8,1004,tt3091254,soul,2013.0,112.0,41 min. or longer,Taiwan,NaN,NaN,NaN,...,0,other fiction,TIFF,Vanguard,2013,0.0,f3349,2013,TIFF,Vanguard
9,1005,tt2919050,soul food stories,2013.0,60.0,41 min. or longer,Bulgaria,Finland,NaN,NaN,...,1,other documentary,IDFA,IDFA Competition for Mid-Length Documentary,2013,0.0,f6233,2013,IDFA,IDFA Competition for Mid-Length Documentary
